In [1]:
import numpy as np
import json
import re

In [2]:
with open("Biobank_Bounding_Boxes_2021_9_9_mobilenet_v3.json","r") as f:
    networkOutput = json.load(f)

with open("UK Biobank large test n19695.json","r") as f:
    targetData = json.load(f)

In [6]:
targets = [float(x["t1_slice_location"]) for x in targetData]
eids = [str(x["id"]) for x in targetData]
targetDict = dict(zip(eids,targets))

trainEids = np.random.choice(eids,size=int(len(eids)*0.9),replace=False)
testEids = [x for x in eids if x not in trainEids]

trainTarget = np.array([targetDict[eid] for eid in trainEids])
testTarget = np.array([targetDict[eid] for eid in testEids])

In [10]:
keysOI = ["Liver","Body","Lung","Heart","Kidney","Spleen","IVC","Aorta"]
trainInput = np.zeros((len(trainEids),(len(keysOI)+2)*4))
testInput = np.zeros((len(testEids),(len(keysOI)+2)*4))

for i,eid in enumerate(trainEids):
    try:
        bBoxes = networkOutput["{}_20204_2_0".format(eid)]
    except:
        continue

    bBKeys = list(bBoxes.keys())
    j = 0
    for kOI in keysOI:
        patt = str("{}\_\d+".format(kOI))
        numOrgs = []
        
        for bb in bBKeys:
            org = re.findall(patt,bb)
            numOrgs.extend(org)

        maxPred = 0.0
        predAcc = np.array([x[4] for x in numOrgs])
        numOrgs = np.array(numOrgs)

        if kOI in ["Lung","Kidney"]:
            topPred = predAcc.argsort()[-2:]
            tempKeys = numOrgs[topPred]
            tempBbox = np.array([bBoxes[tK][:4] for tK in tempKeys])
            tempBbox = tempBbox.flatten()
            trainInput[i,j:j+(4*len(tempKeys))] = tempBbox

            j += 8
        else:
            try:
                topPred = predAcc.argsort()[-1:]
                tempKey = numOrgs[topPred.astype(int)][0]
                tempBbox = bBoxes[tempKey][:4]
                trainInput[i,j:j+4] = tempBbox
            except IndexError:
                pass

            j += 4

for i,eid in enumerate(testEids):
    try:
        bBoxes = networkOutput["{}_20204_2_0".format(eid)]
    except:
        continue
    
    bBKeys = list(bBoxes.keys())
    j = 0
    for kOI in keysOI:
        patt = str("{}\_\d+".format(kOI))
        numOrgs = []
        
        for bb in bBKeys:
            org = re.findall(patt,bb)
            numOrgs.extend(org)

        maxPred = 0.0
        predAcc = np.array([x[4] for x in numOrgs])
        numOrgs = np.array(numOrgs)

        if kOI in ["Lung","Kidney"]:
            topPred = predAcc.argsort()[-2:]
            tempKeys = numOrgs[topPred]
            tempBbox = np.array([bBoxes[tK][:4] for tK in tempKeys])
            tempBbox = tempBbox.flatten()
            testInput[i,j:j+(4*len(tempKeys))] = tempBbox

            j += 8
        else:
            try:
                topPred = predAcc.argsort()[-1:]
                tempKey = numOrgs[topPred.astype(int)][0]
                tempBbox = bBoxes[tempKey][:4]
                testInput[i,j:j+4] = tempBbox
            except IndexError:
                pass

            j += 4

In [11]:
print(trainTarget.shape,trainInput.shape)

(17725,) (17725, 40)


In [12]:
# Ratios...
def volume(inp):
    vol = np.zeros((inp.shape[0],inp.shape[1]//4))
    for i,arr in enumerate(inp):
        for j in range(0,inp.shape[1],4):
            vol[i,j//4] = (arr[j+2]-arr[j])*(arr[j+3]-arr[j+1])
    return vol

def centroid(inp):
    centroid = np.zeros((inp.shape[0],inp.shape[1]//4,2))
    for i,arr in enumerate(inp):
        for j in range(0,inp.shape[1],4):
            centroid[i,j//4,0] = arr[j] + (arr[j+2]-arr[j])//2
            centroid[i,j//4,1] = arr[j+1] + (arr[j+3]-arr[j+1])//2
    return centroid

In [13]:
trainVol = volume(trainInput)
testVol = volume(testInput)

trainCentroid = centroid(trainInput)
testCentroid = centroid(testInput)

print(trainVol.shape,trainCentroid.shape)

(17725, 10) (17725, 10, 2)


In [14]:
# wrongLocationList = []
# with open("reasons\ct1-1-Wrong_location","r") as f:
#     for line in f:
#         wrongLocationList.append(line.strip())

# for dct in targetData:
#     if "wrongslice" in str(dct["Tags"]):
#         wrongLocationList.append(dct["id"])

# print(len(wrongLocationList))
# fullLocList = []
# fullLocDict = {}

# allEids = np.concatenate((trainEids,testEids))

# for eid in allEids:
#     if eid in wrongLocationList:
#         fullLocList.append(True)
#         fullLocDict[eid] = True
#     else:
#         fullLocList.append(False)
#         fullLocDict[eid] = False

# classDict = {0:"Liver",1:"Body",2:"Lung",3:"Lung",4:"Heart",5:"Kidney",6:"Kidney",7:"Spleen",8:"IVC",9:"Aorta"}
# dfDict = {"X":[],"Y":[],"Class":[],"Wrongslice":[]}

# centroidsFull = np.concatenate((trainCentroid,testCentroid))

# print(centroidsFull.shape)
# for i in range(centroidsFull.shape[0]):
#     for j in range(centroidsFull.shape[1]):
#         dfDict["X"].append(centroidsFull[i,j,0])
#         dfDict["Y"].append(centroidsFull[i,j,1])
#         dfDict["Class"].append(classDict[j])
#         dfDict["Wrongslice"] = fullLocDict[allEids[i]]

# df = pandas.DataFrame(dfDict)

# subDF = df[df["Class"] == "Body"]

# sns.scatterplot(data=subDF,x="X",y="Y",hue="Wrongslice")
# plt.show()

# print(df)

In [15]:
def vol_2_dict(vol,eids):
    newVol = {}
    keysOI = ["Liver","Body","Lung","Heart","Kidney","Spleen","IVC","Aorta"]
    for eid,arr in zip(eids,vol):
        tempDict = {}

        i = 0
        j = 0
        while i < arr.shape[0]:

            if i == 2 or i == 5:
                val = np.mean(arr[i:i+1])
                if val < 0.01:
                    val = np.nan
                tempDict[keysOI[j]] = val
                i += 2
            else:
                val = arr[i]
                if val < 0.01:
                    val = np.nan
                tempDict[keysOI[j]] = val
                i += 1

            j += 1

        newVol[eid] = tempDict

    return newVol

def cent_2_dict(cent,eids):
    newCent = {}
    keysOI = ["Liver","Body","Lung","Lung","Heart","Kidney","Kidney","Spleen","IVC","Aorta"]
    for eid,arr in zip(eids,cent):
        newCent[eid] = {}

        for c,org in zip(arr,keysOI):
            newCent[eid][org] = c

    return newCent        

In [16]:
trainVolDict = vol_2_dict(trainVol,trainEids)
testVolDict = vol_2_dict(testVol,testEids)

trainCentDict = cent_2_dict(trainCentroid,trainEids)
testCentDict = cent_2_dict(testCentroid,trainEids)

print(trainVolDict[trainEids[0]],trainCentDict[trainEids[0]])

{'Liver': 18986.063307704404, 'Body': 40739.63524892635, 'Lung': nan, 'Heart': nan, 'Kidney': 2287.679886834696, 'Spleen': nan, 'IVC': 328.45732543291524, 'Aorta': 567.6893249638379} {'Liver': array([160.30152893, 145.46839905]), 'Body': array([188.15102768, 146.88304138]), 'Lung': array([0., 0.]), 'Heart': array([0., 0.]), 'Kidney': array([0., 0.]), 'Spleen': array([0., 0.]), 'IVC': array([171.00770569, 143.39323425]), 'Aorta': array([205.31214905, 142.82992554])}


In [25]:
import pandas
import seaborn as sns
import matplotlib.pyplot as plt

dfVol = pandas.DataFrame(trainVolDict)
dfVol = dfVol.T
dfVol

,Liver,Body,Lung,Heart,Kidney,Spleen,IVC,Aorta
1513651,18986.063308,40739.635249,NaN,NaN,2287.679887,NaN,328.457325,567.689325
2532948,21771.029460,70052.845062,15901.030081,2982.782096,NaN,3464.972379,NaN,688.923507
2827805,24321.851593,68093.830558,NaN,NaN,NaN,4292.074988,265.495184,551.939067
3141338,17674.419430,39132.299532,NaN,NaN,836.093846,3898.598946,289.336655,498.314448
4149230,16758.558271,68514.409973,NaN,NaN,2852.926477,NaN,1722.659465,607.447442
...,...,...,...,...,...,...,...,...
2412031,8180.958354,57519.395357,10740.238512,6496.909669,NaN,NaN,NaN,562.272667
2556362,24649.707270,76816.398812,13064.838534,1439.784045,981.288708,7204.581868,512.805728,1179.736218
4739137,23578.213897,48108.286647,NaN,NaN,1449.148630,1508.166717,352.960449,956.981467
5016804,23719.185426,65447.911854,8156.286493,NaN,3036.277271,3058.795611,232.312011,807.461984


In [26]:
dfCent = pandas.DataFrame(trainCentDict)
dfCent = dfCent.T
dfCent

,Liver,Body,Lung,Heart,Kidney,Spleen,IVC,Aorta
1513651,"[160.30152893066406, 145.46839904785156]","[188.15102767944336, 146.88304138183594]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[171.00770568847656, 143.3932342529297]","[205.31214904785156, 142.82992553710938]"
2532948,"[175.90857696533203, 148.303955078125]","[210.24076080322266, 149.2196807861328]","[280.2966003417969, 150.53568267822266]","[254.2986602783203, 109.49903106689453]","[0.0, 0.0]","[283.290771484375, 197.95803833007812]","[0.0, 0.0]","[224.60501098632812, 180.86439514160156]"
2827805,"[162.1686019897461, 163.63349151611328]","[198.83124542236328, 158.00767135620117]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[279.96246337890625, 206.93829345703125]","[166.52613830566406, 176.39613342285156]","[211.63284301757812, 184.62380981445312]"
3141338,"[163.6893310546875, 173.58863830566406]","[193.21295166015625, 173.25603485107422]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[263.9447326660156, 199.3618621826172]","[171.9451904296875, 177.89703369140625]","[200.97869873046875, 178.80247497558594]"
4149230,"[110.25925064086914, 160.95670318603516]","[185.84114837646484, 158.5361671447754]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[153.08985900878906, 150.05899810791016]","[154.07257080078125, 156.51998901367188]"
...,...,...,...,...,...,...,...,...
2412031,"[138.85800170898438, 153.73505401611328]","[197.36892318725586, 147.98040008544922]","[218.57510375976562, 156.32669830322266]","[213.82723999023438, 119.19371795654297]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[214.7306671142578, 164.9340362548828]"
2556362,"[135.7345085144043, 161.02804565429688]","[188.3412208557129, 154.32054138183594]","[0.0, 0.0]","[208.85067749023438, 182.77381896972656]","[259.8242645263672, 173.25015258789062]","[268.02601623535156, 186.3166046142578]","[168.8673553466797, 159.81222534179688]","[208.34683227539062, 181.77149963378906]"
4739137,"[175.61549377441406, 149.98672485351562]","[203.89550018310547, 147.4493522644043]","[0.0, 0.0]","[0.0, 0.0]","[242.09986877441406, 185.4669952392578]","[266.23060607910156, 151.68411254882812]","[175.5264129638672, 153.03646850585938]","[261.9993591308594, 149.0515594482422]"
5016804,"[154.5901107788086, 165.0034942626953]","[202.98369598388672, 164.7304458618164]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[273.8771667480469, 213.70184326171875]","[181.32518005371094, 163.65528869628906]","[221.50405883789062, 165.85865783691406]"


In [27]:
trainRatioDict = {}
for eid in trainVolDict.keys():
    tempDict = {}
    for org in keysOI:
        tempDict[org] = trainVolDict[eid][org]/trainVolDict[eid]["Body"]
    trainRatioDict[eid] = tempDict

ratioDf = pandas.DataFrame(trainRatioDict)
ratioDf = ratioDf.T
ratioDf

,Liver,Body,Lung,Heart,Kidney,Spleen,IVC,Aorta
1513651,0.466034,1.0,NaN,NaN,0.056154,NaN,0.008062,0.013935
2532948,0.310780,1.0,0.226986,0.042579,NaN,0.049462,NaN,0.009834
2827805,0.357181,1.0,NaN,NaN,NaN,0.063032,0.003899,0.008106
3141338,0.451658,1.0,NaN,NaN,0.021366,0.099626,0.007394,0.012734
4149230,0.244599,1.0,NaN,NaN,0.041640,NaN,0.025143,0.008866
...,...,...,...,...,...,...,...,...
2412031,0.142230,1.0,0.186724,0.112952,NaN,NaN,NaN,0.009775
2556362,0.320891,1.0,0.170079,0.018743,0.012774,0.093790,0.006676,0.015358
4739137,0.490107,1.0,NaN,NaN,0.030123,0.031349,0.007337,0.019892
5016804,0.362413,1.0,0.124623,NaN,0.046392,0.046736,0.003550,0.012337


In [28]:
wrongLocationList = []
with open("reasons\ct1-1-Wrong_location","r") as f:
    for line in f:
        wrongLocationList.append(line.strip())

for dct in targetData:
    if "wrongslice" in str(dct["Tags"]):
        wrongLocationList.append(dct["id"])
    elif "Wrongslice" in str(dct["Tags"]):
        wrongLocationList.append(dct["id"])

fullLocList = []
fullLocDict = {}
for eid in trainEids:
    if eid in wrongLocationList:
        fullLocList.append(True)
        fullLocDict[eid] = True
    else:
        fullLocList.append(False)
        fullLocDict[eid] = False

print(np.sum(fullLocList))

40


In [29]:
newDfVol = []
for eid in trainVolDict.keys():
    volRatio = trainVolDict[eid]["Liver"]/trainVolDict[eid]["Body"]
    newDfVol.append({"Eid":eid,"Liver to Body ratio":volRatio,"Wrong Location":fullLocDict[eid]})

newDfVol = pandas.DataFrame(newDfVol)
newDfVol.to_json("vol_ratio.json")
newDfVol

,Eid,Liver to Body ratio,Wrong Location
0,1513651,0.466034,False
1,2532948,0.310780,False
2,2827805,0.357181,False
3,3141338,0.451658,False
4,4149230,0.244599,False
...,...,...,...
17720,2412031,0.142230,True
17721,2556362,0.320891,False
17722,4739137,0.490107,False
17723,5016804,0.362413,False


In [30]:
newDfCent = []
for eid in trainCentDict.keys():
    newDfCent.append({"Eid":eid,"X":trainCentDict[eid]["Liver"][0],"Y":trainCentDict[eid]["Liver"][1],"Wrong Location":fullLocDict[eid]})

newDfCent = pandas.DataFrame(newDfCent)
newDfCent.to_json("centroid.json")
newDfCent

,Eid,X,Y,Wrong Location
0,1513651,160.301529,145.468399,False
1,2532948,175.908577,148.303955,False
2,2827805,162.168602,163.633492,False
3,3141338,163.689331,173.588638,False
4,4149230,110.259251,160.956703,False
...,...,...,...,...
17720,2412031,138.858002,153.735054,True
17721,2556362,135.734509,161.028046,False
17722,4739137,175.615494,149.986725,False
17723,5016804,154.590111,165.003494,False


In [31]:
modDf = []

for kOI in keysOI:
    tempArr = dfVol[kOI].to_list()
    tempOrgs = [kOI for _ in tempArr]
    modDf.extend(list(zip(tempArr,tempOrgs,trainTarget,trainEids,fullLocList)))

modDf = np.array(modDf)
modDf = {"Volume":modDf[:,0],"Organ":modDf[:,1],"Slice Location":modDf[:,2],"Eid":modDf[:,3],"Wrong Location":modDf[:,4]}
modDf = pandas.DataFrame(modDf)
modDf.to_json("vol_org_sL.json")

In [35]:
modDf = []

for kOI in keysOI:
    tempArr = dfCent[kOI].to_list()
    tempArr = np.array(tempArr)
    tempOrgs = [kOI for _ in range(tempArr.shape[0])]
    tempX = list(tempArr[:,0])
    tempY = list(tempArr[:,1])
    modDf.extend(list(zip(tempX,tempY,tempOrgs,trainTarget,trainEids,fullLocList)))

modDf = np.array(modDf)
modDf = {"X":modDf[:,0],"Y":modDf[:,1],"Organ":modDf[:,2],"Slice Location":modDf[:,3],"Eid":modDf[:,4],"Wrong Location":modDf[:,5]}
modDf = pandas.DataFrame(modDf)
modDf.to_json("centroid_org_sL.json")

In [38]:
modDf = []

body = dfCent["Body"].to_list()
body = np.array(body)

for kOI in keysOI:
    tempArr = dfCent[kOI].to_list()
    tempArr = np.array(tempArr)
    tempOrgs = [kOI for _ in range(tempArr.shape[0])]
    
    if kOI != "Body":
        tempX = list(tempArr[:,0]-body[:,0])
        tempY = list(tempArr[:,1]-body[:,1])
    
    else:
        tempX = list(body[:,0])
        tempY = list(body[:,1])

    modDf.extend(list(zip(tempX,tempY,tempOrgs,trainTarget,trainEids,fullLocList)))

modDf = np.array(modDf)
modDf = {"X":modDf[:,0],"Y":modDf[:,1],"Organ":modDf[:,2],"Slice Location":modDf[:,3],"Eid":modDf[:,4],"Wrong Location":modDf[:,5]}
modDf = pandas.DataFrame(modDf)
modDf.to_json("centroid_org_sL_norm.json")

In [36]:
modDf = []

for kOI in keysOI:
    tempArr = ratioDf[kOI].to_list()
    tempOrgs = [kOI for _ in tempArr]
    modDf.extend(list(zip(tempArr,tempOrgs,trainTarget,trainEids,fullLocList)))

modDf = np.array(modDf)
modDf = {"Norm. Volume":modDf[:,0],"Organ":modDf[:,1],"Slice Location":modDf[:,2],"Eid":modDf[:,3],"Wrong Location":modDf[:,4]}
modDf = pandas.DataFrame(modDf)
modDf.to_json("vol_org_sL_norm.json")

In [37]:
print(modDf)

                Norm. Volume  Organ Slice Location      Eid Wrong Location
0        0.46603419966075327  Liver        -218.25  1513651          False
1         0.3107800895316046  Liver        -231.75  2532948          False
2         0.3571814273584301  Liver        -272.25  2827805          False
3        0.45165808402245017  Liver        -282.75  3141338          False
4        0.24459903073570707  Liver        -267.75  4149230          False
...                      ...    ...            ...      ...            ...
141795  0.009775357755359936  Aorta        -200.25  2412031           True
141796   0.01535786935989572  Aorta        -164.25  2556362          False
141797  0.019892237562257383  Aorta        -222.75  4739137          False
141798  0.012337475118934897  Aorta        -278.25  5016804          False
141799  0.008590380346343254  Aorta        -186.75  4385667          False

[141800 rows x 5 columns]


In [78]:
import csv

subDf = modDf[modDf["Organ"] == "Heart"]
subDf = subDf.astype({"Norm. Volume":float})
subDf = subDf[subDf["Norm. Volume"] > 0.5]
subDf = subDf[subDf["Wrong Location"] == "False"]

frCNNTrainEids = []
with open("data_new.csv","r") as f:
    csvObj = csv.reader(f)
    for line in csvObj:
        frCNNTrainEids.append(line[0][:7])

trainEids = pandas.read_csv("train_new_format.csv",header=None)
valEids = pandas.read_csv("val_new_format.csv",header=None)

trainEids = list(trainEids[0])
trainEids.extend(list(valEids[0]))
trainEids = [x[:7] for x in trainEids]

subEids = subDf["Eid"].to_list()
subEids = [x for x in subEids if x not in frCNNTrainEids]
print(len(subEids))
subEids = [x for x in subEids if x not in trainEids]
print(len(subEids))

136
136


In [79]:
import matplotlib.patches as patches

plt.rcParams["figure.figsize"] = [6.4*1,4.8*1]

bbNames = ["Liver","Body","Lung","Heart","Kidney","Spleen","IVC","Aorta","Cyst"]
maxCount = {"Liver":1,"Body":1,"Lung":2,"Heart":1,"Kidney":2,"Spleen":1,"IVC":1,"Aorta":1,"Cyst":10}
colours = [plt.cm.Set1(i) for i in range(9)]
colourDict = dict(zip(bbNames,colours))

for eid in subEids:
    orgCnt = {bbN:0 for bbN in bbNames}
    arr = np.load("C:/fully_split_data/{}_20204_2_0.npy".format(eid))
    bBoxes = networkOutput["{}_20204_2_0".format(eid)]
    
    # fig, ax = plt.subplots(1,7)
    # for i in range(7):
    #     ax[i].imshow(arr[:,:,i])
    #     ax[i].axis("off")
    #     for k in bBoxes.keys():
    #         coord = bBoxes[k]
    #         org = k[:-2]
    #         if coord[-1] > 0.9:
    #             rect = patches.Rectangle((coord[0],coord[1]),coord[2]-coord[0],coord[3]-coord[1],linewidth=1,edgecolor=colourDict[org],facecolor='none')
    #             ax[i].text(coord[0]-2,coord[1]-2,k[:-2],color=colourDict[org])
    #             ax[i].add_patch(rect)
    
    fig,ax = plt.subplots(1,1)
    ax.imshow(arr[:,:,0],vmax=100,cmap="gray")
    ax.axis("off")
    for k in bBoxes.keys():
        coord = bBoxes[k]
        org = k[:-2]
        if coord[-1] > 0.5:
            if orgCnt[org] < maxCount[org]:
                rect = patches.Rectangle((coord[0],coord[1]),coord[2]-coord[0],coord[3]-coord[1],linewidth=2,edgecolor=colourDict[org],facecolor='none')
                ax.text(coord[0]-2,coord[1]-2,k[:-2],color=colourDict[org])
                ax.add_patch(rect)
                orgCnt[org] += 1

    plt.savefig("./Figures/{}.png".format(eid))
    plt.close("all")